In [47]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [48]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-73hw4z9q
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-73hw4z9q
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 5741c522547756ac4bb7a16df32106a15efb8a57
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [49]:
%load_ext nvcc4jupyter

The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


In [50]:
!git clone https://github.com/nothings/stb.git
!cp stb/stb_image.h /usr/local/include/

fatal: destination path 'stb' already exists and is not an empty directory.


In [51]:
!pip install wget
!wget https://raw.githubusercontent.com/nothings/stb/master/stb_image_write.h

--2024-04-17 16:18:08--  https://raw.githubusercontent.com/nothings/stb/master/stb_image_write.h
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71221 (70K) [text/plain]
Saving to: ‘stb_image_write.h.2’

stb_image_write.h.2 100%[===================>]  69.55K  --.-KB/s    in 0.003s  

2024-04-17 16:18:08 (23.9 MB/s) - ‘stb_image_write.h.2’ saved [71221/71221]



# Final =========================>


In [56]:
%%writefile K1_2_31_2_33.cu
//Norhan_Reda_Abdelwahed_2_31
//Hoda_Gamal_Hamouda_2_33

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <iostream>
#include <iomanip>
#include <fstream>
#include <vector>
#define STB_IMAGE_IMPLEMENTATION
#include <stdio.h>
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"
#include <string>
#include <filesystem>
#include <cuda_runtime.h>
using namespace std;

// Kernel definition
__global__ void Imgcov3d(int batchSize, float *input, float* output ,unsigned int width, unsigned int height , const float* __restrict__ mask, int maskdim){

    int outRow = blockIdx.y * blockDim.y + threadIdx.y;
    int outCol = blockIdx.x * blockDim.x + threadIdx.x;
    int imageIndex = blockIdx.z * blockDim.z + threadIdx.z;

    if(outRow < height && outCol < width && imageIndex < batchSize) {

      float sum = 0;

        for(int maskRow = 0; maskRow < maskdim; ++maskRow){
           for(int maskCol = 0; maskCol < maskdim; ++maskCol){
              for(int maskdep = 0; maskdep < 3; ++maskdep){

                  int inRow = outRow -(maskdim/2) + maskRow;
                  int inCol = outCol -(maskdim/2) + maskCol;

                  if(inRow < height && inRow >= 0 && inCol < width && inCol >= 0){
                      sum += mask[maskdep * maskdim * maskdim + maskRow * maskdim + maskCol]
                      *input[imageIndex * width * height * 3 + maskdep * width * height + inRow * width + inCol];
                  }
             }
          }
        }
        if(sum < 0){
          sum = 0;
        } else if (sum > 255){
          sum = 255;
        }
        output[imageIndex * width * height + outRow * width + outCol] = sum;
    }

}

int main(int argc, char *argv[]) {

    string folderPath = argv[1];
    string out_path = argv[2];
    int batch_size = stoi(argv[3]);
    string mask_path = argv[4];

    int maskdim;

    // Open the file for reading
    std::ifstream inputFile(mask_path);

    // Check if the file is opened successfully
    if (!inputFile.is_open()) {
        std::cerr << "Error opening file: " <<mask_path  << std::endl;
        return 1; // Exit with an error code
    }

     inputFile >> maskdim ;
     // cout<<"maskdim "<<maskdim<<endl;

    float *h_A, *h_B,*R,*G,*B;
    int rows1,cols1, comp;

    float *h_images;
    int img_index = 0;

    for (const auto& entry : std::filesystem::directory_iterator(folderPath)) {

        if (entry.is_regular_file()) {
            std::string filePath = entry.path().string();

            unsigned char *data = stbi_load(filePath.c_str(), &cols1, &rows1, &comp, 0);

            if(img_index == 0){
              h_images = (float*)malloc(sizeof(float) * batch_size * cols1 * rows1 * comp);
            }

            if (data) {

                printf("cols %d\n", cols1);
                printf("rows %d\n", rows1);
                printf("image %s\n", filePath.c_str());

                h_A = (float*)malloc(sizeof(float) *  rows1 * cols1* comp);
                R = (float*)malloc(sizeof(float) *  rows1 * cols1);
                G = (float*)malloc(sizeof(float) *  rows1 * cols1);
                B = (float*)malloc(sizeof(float) *  rows1 * cols1);

                int k = 0;
                for(int i=0;i<rows1 * cols1* comp;i=i+3)
                {
                  R[k] =  static_cast<float>(data[i]);
                  k=k+1;
                }
                k=0;
                for(int i=1;i<rows1 * cols1* comp;i=i+3)
                {
                  G[k] =  static_cast<float>(data[i]);
                  k=k+1;
                }
                k=0;
                for(int i=2;i<rows1 * cols1* comp;i=i+3)
                {
                  B[k] =  static_cast<float>(data[i]);
                  k=k+1;
                }


                memcpy(h_A, R, rows1*cols1 * sizeof(float));
                memcpy(h_A +rows1*cols1 , G, rows1*cols1 * sizeof(float));
                memcpy(h_A+rows1*cols1+rows1*cols1, B, rows1*cols1 * sizeof(float));

                memcpy(h_images + img_index * rows1 * cols1 * 3,                     R, rows1 * cols1 * sizeof(float));
                memcpy(h_images + img_index * rows1 * cols1 * 3 + rows1 * cols1,     G, rows1 * cols1 * sizeof(float));
                memcpy(h_images + img_index * rows1 * cols1 * 3 + rows1 * cols1 * 2, B, rows1 * cols1 * sizeof(float));

                img_index++;

             }
             else {
                // Failed to load the image
                std::cerr << "Failed to load image: " << filePath << std::endl;
            }

           }
      }


      float *d_images_temp = (float*)malloc(sizeof(float) * batch_size * rows1 * cols1 * comp);
      for(int i=0; i<batch_size; i++){
        memcpy(d_images_temp + i * rows1 * cols1 *comp , h_images + i * rows1 * cols1 *comp, rows1 * cols1 *comp * sizeof(float));
      }


    float *d_images;
    cudaMalloc((void**)&d_images, batch_size * rows1 * cols1 * comp * sizeof(float));
    cudaMemcpy(d_images, d_images_temp, batch_size * rows1 * cols1 * comp * sizeof(float), cudaMemcpyHostToDevice);


    h_B =(float*)malloc(sizeof(float) * maskdim* maskdim*comp);

    float *d_outs;
    cudaMalloc((void**)&d_outs, batch_size * rows1 * cols1 * sizeof(float));

    float *h_outs = (float*)malloc(sizeof(float) * batch_size * rows1 * cols1);


    float *h_R, *h_G,*h_BB;
    h_R =(float*)malloc(sizeof(float) * maskdim* maskdim);
    h_G =(float*)malloc(sizeof(float) * maskdim* maskdim);
    h_BB =(float*)malloc(sizeof(float) * maskdim* maskdim);
    // Read mask elements
    for (int i = 0; i < maskdim; ++i) {
        for (int j = 0; j < maskdim; ++j) {

            int index =  i * maskdim + j;
            inputFile >>  h_R[index];
            h_R[index]=h_R[index];
            h_G[index]=h_R[index];
            h_BB[index]=h_R[index];

        }
    }

    memcpy(h_B, h_R,  maskdim* maskdim * sizeof(float));
    memcpy(h_B + maskdim* maskdim , h_G, maskdim* maskdim * sizeof(float));
    memcpy(h_B+maskdim* maskdim+maskdim* maskdim,  h_BB, maskdim* maskdim * sizeof(float));


    // Copy data from host to device
    //cudaMemcpyToSymbol(mask, h_B, maskdim * maskdim *comp * sizeof(float), 0, cudaMemcpyHostToDevice);

    float * d_B ;
    cudaMalloc((void**)&d_B, maskdim *  maskdim *comp * sizeof(float));
    cudaMemcpy(d_B, h_B,   maskdim *  maskdim *comp * sizeof(float), cudaMemcpyHostToDevice);

    // Kernel invocation
    dim3 threadsPerBlock(32,32,1);
    dim3 numBlocks((cols1-1) / threadsPerBlock.x + 1, (rows1-1) / threadsPerBlock.y + 1, batch_size);
    Imgcov3d<<<numBlocks, threadsPerBlock>>>(batch_size,d_images, d_outs,cols1,rows1,d_B, maskdim);

    cudaMemcpy(h_outs, d_outs, batch_size * rows1 * cols1 * sizeof(float), cudaMemcpyDeviceToHost);

    unsigned char* h_char = (unsigned char*)malloc(sizeof(unsigned char) * rows1 * cols1);

    for(int k=0;k<batch_size;k++){

      for(int i=0;i<rows1*cols1;i++){
         h_char[i]=static_cast<unsigned char>(h_outs[k*rows1*cols1+i]);
      }
      string filename = "./"+out_path+"/"+"image" + std::to_string(k) + ".jpg";
      stbi_write_jpg(filename.c_str(), cols1, rows1, 1, h_char, 100);
    }

    // Free device memory
    cudaFree(d_images);
    cudaFree(d_B);
    cudaFree(d_outs);

    // Free host memory
    free(h_A);
    free(h_B);
    free(R);
    free(G);
    free(B);
    free(h_images);
    free(d_images_temp);
    free(h_outs);
    free(h_BB);
    free(h_G);
    free(h_R);
    free(h_char);

  return 0;
}

Overwriting K1_2_31_2_33.cu


In [57]:
!chmod +x K1_2_31_2_33.cu
!nvcc  K1_2_31_2_33.cu -o K1_2_31_2_33

In [58]:
!nvprof ./K1_2_31_2_33 "in_255" "out1" 4 "mask_3.txt"

cols 255
rows 255
image in_255/you.jpg
cols 255
rows 255
image in_255/warda.jpg
cols 255
rows 255
image in_255/cat.jpg
cols 255
rows 255
image in_255/camel.jpg
==9502== NVPROF is profiling process 9502, command: ./K1_2_31_2_33 in_255 out1 4 mask_3.txt
==9502== Profiling application: ./K1_2_31_2_33 in_255 out1 4 mask_3.txt
==9502== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   78.57%  540.82us         2  270.41us     992ns  539.83us  [CUDA memcpy HtoD]
                   12.01%  82.653us         1  82.653us  82.653us  82.653us  [CUDA memcpy DtoH]
                    9.43%  64.895us         1  64.895us  64.895us  64.895us  Imgcov3d(int, float*, float*, unsigned int, unsigned int, float const *, int)
      API calls:   97.75%  121.61ms         3  40.537ms  9.5740us  121.43ms  cudaMalloc
                    1.53%  1.9007ms         3  633.57us  16.614us  1.1222ms  cudaMemcpy
                    0.29%  361.04us         

In [68]:
!nvprof ./K1_2_31_2_33 "in_255" "out1" 4 "mask_5.txt"

cols 255
rows 255
image in_255/you.jpg
cols 255
rows 255
image in_255/warda.jpg
cols 255
rows 255
image in_255/cat.jpg
cols 255
rows 255
image in_255/camel.jpg
==12887== NVPROF is profiling process 12887, command: ./K1_2_31_2_33 in_255 out1 4 mask_5.txt
==12887== Profiling application: ./K1_2_31_2_33 in_255 out1 4 mask_5.txt
==12887== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   69.42%  532.82us         2  266.41us     992ns  531.83us  [CUDA memcpy HtoD]
                   19.30%  148.12us         1  148.12us  148.12us  148.12us  Imgcov3d(int, float*, float*, unsigned int, unsigned int, float const *, int)
                   11.29%  86.622us         1  86.622us  86.622us  86.622us  [CUDA memcpy DtoH]
      API calls:   97.56%  97.830ms         3  32.610ms  7.1460us  97.704ms  cudaMalloc
                    1.73%  1.7330ms         3  577.68us  12.612us  1.0262ms  cudaMemcpy
                    0.34%  336.72us     

In [69]:
!nvprof ./K1_2_31_2_33 "in_255" "out1" 4 "mask_7.txt"

cols 255
rows 255
image in_255/you.jpg
cols 255
rows 255
image in_255/warda.jpg
cols 255
rows 255
image in_255/cat.jpg
cols 255
rows 255
image in_255/camel.jpg
==13010== NVPROF is profiling process 13010, command: ./K1_2_31_2_33 in_255 out1 4 mask_7.txt
==13010== Profiling application: ./K1_2_31_2_33 in_255 out1 4 mask_7.txt
==13010== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   58.16%  542.26us         2  271.13us  1.0240us  541.23us  [CUDA memcpy HtoD]
                   29.28%  272.95us         1  272.95us  272.95us  272.95us  Imgcov3d(int, float*, float*, unsigned int, unsigned int, float const *, int)
                   12.56%  117.12us         1  117.12us  117.12us  117.12us  [CUDA memcpy DtoH]
      API calls:   97.03%  92.530ms         3  30.843ms  8.0690us  92.387ms  cudaMalloc
                    2.17%  2.0647ms         3  688.24us  14.605us  1.3420ms  cudaMemcpy
                    0.38%  357.62us     

In [70]:
!nvprof ./K1_2_31_2_33 "in_255" "out1" 4 "mask_9.txt"

cols 255
rows 255
image in_255/you.jpg
cols 255
rows 255
image in_255/warda.jpg
cols 255
rows 255
image in_255/cat.jpg
cols 255
rows 255
image in_255/camel.jpg
==13117== NVPROF is profiling process 13117, command: ./K1_2_31_2_33 in_255 out1 4 mask_9.txt
==13117== Profiling application: ./K1_2_31_2_33 in_255 out1 4 mask_9.txt
==13117== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   49.61%  525.33us         2  262.67us  1.0240us  524.31us  [CUDA memcpy HtoD]
                   41.49%  439.25us         1  439.25us  439.25us  439.25us  Imgcov3d(int, float*, float*, unsigned int, unsigned int, float const *, int)
                    8.90%  94.237us         1  94.237us  94.237us  94.237us  [CUDA memcpy DtoH]
      API calls:   97.19%  96.887ms         3  32.296ms  7.9610us  96.759ms  cudaMalloc
                    2.03%  2.0227ms         3  674.25us  13.387us  1.3100ms  cudaMemcpy
                    0.36%  357.22us     

In [71]:
!nvprof ./K1_2_31_2_33 "in_255" "out1" 4 "mask_11.txt"

cols 255
rows 255
image in_255/you.jpg
cols 255
rows 255
image in_255/warda.jpg
cols 255
rows 255
image in_255/cat.jpg
cols 255
rows 255
image in_255/camel.jpg
==13202== NVPROF is profiling process 13202, command: ./K1_2_31_2_33 in_255 out1 4 mask_11.txt
==13202== Profiling application: ./K1_2_31_2_33 in_255 out1 4 mask_11.txt
==13202== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   50.15%  647.15us         1  647.15us  647.15us  647.15us  Imgcov3d(int, float*, float*, unsigned int, unsigned int, float const *, int)
                   40.87%  527.41us         2  263.70us  1.0560us  526.35us  [CUDA memcpy HtoD]
                    8.98%  115.84us         1  115.84us  115.84us  115.84us  [CUDA memcpy DtoH]
      API calls:   97.01%  97.763ms         3  32.588ms  5.5310us  97.634ms  cudaMalloc
                    2.25%  2.2682ms         3  756.06us  13.584us  1.5370ms  cudaMemcpy
                    0.34%  347.15us   

In [72]:
!nvprof ./K1_2_31_2_33 "in_255" "out1" 4 "mask_13.txt"

cols 255
rows 255
image in_255/you.jpg
cols 255
rows 255
image in_255/warda.jpg
cols 255
rows 255
image in_255/cat.jpg
cols 255
rows 255
image in_255/camel.jpg
==13279== NVPROF is profiling process 13279, command: ./K1_2_31_2_33 in_255 out1 4 mask_13.txt
==13279== Profiling application: ./K1_2_31_2_33 in_255 out1 4 mask_13.txt
==13279== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   59.65%  900.10us         1  900.10us  900.10us  900.10us  Imgcov3d(int, float*, float*, unsigned int, unsigned int, float const *, int)
                   34.58%  521.81us         2  260.91us  1.0560us  520.75us  [CUDA memcpy HtoD]
                    5.77%  87.102us         1  87.102us  87.102us  87.102us  [CUDA memcpy DtoH]
      API calls:   96.75%  95.104ms         3  31.701ms  4.8130us  94.988ms  cudaMalloc
                    2.51%  2.4691ms         3  823.04us  13.387us  1.7747ms  cudaMemcpy
                    0.36%  356.73us   

In [73]:
!nvprof ./K1_2_31_2_33 "in_255" "out1" 4 "mask_15.txt"

cols 255
rows 255
image in_255/you.jpg
cols 255
rows 255
image in_255/warda.jpg
cols 255
rows 255
image in_255/cat.jpg
cols 255
rows 255
image in_255/camel.jpg
==13350== NVPROF is profiling process 13350, command: ./K1_2_31_2_33 in_255 out1 4 mask_15.txt
==13350== Profiling application: ./K1_2_31_2_33 in_255 out1 4 mask_15.txt
==13350== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   65.69%  1.1885ms         1  1.1885ms  1.1885ms  1.1885ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, float const *, int)
                   29.72%  537.71us         2  268.86us  1.1200us  536.59us  [CUDA memcpy HtoD]
                    4.59%  83.133us         1  83.133us  83.133us  83.133us  [CUDA memcpy DtoH]
      API calls:   96.45%  96.969ms         3  32.323ms  5.4380us  96.842ms  cudaMalloc
                    2.86%  2.8705ms         3  956.83us  13.738us  2.1107ms  cudaMemcpy
                    0.33%  335.19us   

In [74]:
!nvprof ./K1_2_31_2_33 "in_255" "out1" 4 "mask_edge_5.txt"

cols 255
rows 255
image in_255/you.jpg
cols 255
rows 255
image in_255/warda.jpg
cols 255
rows 255
image in_255/cat.jpg
cols 255
rows 255
image in_255/camel.jpg
==13571== NVPROF is profiling process 13571, command: ./K1_2_31_2_33 in_255 out1 4 mask_edge_5.txt
==13571== Profiling application: ./K1_2_31_2_33 in_255 out1 4 mask_edge_5.txt
==13571== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   66.62%  527.92us         2  263.96us     992ns  526.93us  [CUDA memcpy HtoD]
                   18.69%  148.12us         1  148.12us  148.12us  148.12us  Imgcov3d(int, float*, float*, unsigned int, unsigned int, float const *, int)
                   14.69%  116.38us         1  116.38us  116.38us  116.38us  [CUDA memcpy DtoH]
      API calls:   97.53%  96.424ms         3  32.141ms  7.4830us  96.304ms  cudaMalloc
                    1.77%  1.7457ms         3  581.90us  11.881us  1.0396ms  cudaMemcpy
                    0.35%  342

In [88]:
%%writefile K2_2_31_2_33.cu
//Norhan_Reda_Abdelwahed_2_31
//Hoda_Gamal_Hamouda_2_33

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <iostream>
#include <iomanip>
#include <fstream>
#include <vector>
#define STB_IMAGE_IMPLEMENTATION
#include <stdio.h>
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"
#include <string>
#include <filesystem>
#include <cuda_runtime.h>

using namespace std;

// Kernel definition
__global__ void Imgcov3d(int batchSize, float *input, float* output, unsigned int width,unsigned int height, const float* __restrict__ mask,int maskdim, int shared_width,int shared_height, int shared_depth){

  extern __shared__ float sharedTile [];

    int tx = threadIdx.x;
    int ty = threadIdx.y;

    int tile_x = shared_width - (maskdim - 1);
    int tile_y = shared_height - (maskdim - 1);

    int x_o = blockIdx.x * tile_x + threadIdx.x;
    int y_o = blockIdx.y * tile_y + threadIdx.y;

    int x_i = x_o - maskdim/2;
    int y_i = y_o - maskdim/2;

    int idx = blockIdx.z * blockDim.z + threadIdx.z;

    if (idx >= batchSize){
      return;
    }

  for (int d = 0; d < 3; d++) {

    if (x_i >= 0 && y_i >= 0 && x_i < width && y_i < height){

       sharedTile[idx * shared_width * shared_height * 3 + d * shared_height * shared_width + ty * shared_width + tx] =
        input[idx * width * height * 3 + d * height * width + y_i * width + x_i];
    }
    else{
       sharedTile[idx * shared_width * shared_height * 3 + d * shared_height * shared_width + ty * shared_width + tx] = 0;
    }
  }
    __syncthreads();

    float sum = 0.0;
    if(ty < tile_y && tx < tile_x) {

      for(int y_mask = 0; y_mask < maskdim; y_mask++) {
            for(int x_mask = 0; x_mask < maskdim; x_mask++) {
                  for(int z_mask = 0; z_mask < 3; z_mask++) {
                    sum += mask[z_mask * maskdim * maskdim + y_mask * maskdim + x_mask] *
                    sharedTile[idx * shared_height * shared_width * 3 + z_mask * shared_height * shared_width + (ty + y_mask) * shared_width +(tx + x_mask)];

                }
             }
         }

    if(sum < 0){
      sum = 0;
    } else if (sum > 255){
      sum = 255;
    }

    if(y_o < height && x_o < width)
        output[idx * width * height + y_o * width + x_o] = sum;
    }
}

int main(int argc, char *argv[]) {

    string folderPath = argv[1];
    string out_path = argv[2];
    int batch_size = stoi(argv[3]);
    string mask_path = argv[4];

    int maskdim;

    // Open the file for reading
    std::ifstream inputFile(mask_path);

    // Check if the file is opened successfully
    if (!inputFile.is_open()) {
        std::cerr << "Error opening file: " <<mask_path  << std::endl;
        return 1; // Exit with an error code
    }

     inputFile >> maskdim ;
     cout<<"maskdim "<<maskdim<<endl;

    float *h_A, *h_B,*R,*G,*B;
    int rows1,cols1, comp;

    float *h_images;
    int img_index = 0;

    for (const auto& entry : std::filesystem::directory_iterator(folderPath)) {

        if (entry.is_regular_file()) {
            std::string filePath = entry.path().string();

            unsigned char *data = stbi_load(filePath.c_str(), &cols1, &rows1, &comp, 0);

            if(img_index == 0){
              h_images = (float*)malloc(sizeof(float) * batch_size * cols1 * rows1 * comp);
            }

            if (data) {

                printf("cols %d\n", cols1);
                printf("rows %d\n", rows1);
                printf("image %s\n", filePath.c_str());

                h_A = (float*)malloc(sizeof(float) *  rows1 * cols1* comp);
                R = (float*)malloc(sizeof(float) *  rows1 * cols1);
                G = (float*)malloc(sizeof(float) *  rows1 * cols1);
                B = (float*)malloc(sizeof(float) *  rows1 * cols1);

                int k = 0;
                for(int i=0;i<rows1 * cols1* comp;i=i+3)
                {
                  R[k] =  static_cast<float>(data[i]);
                  k=k+1;
                }
                k=0;
                for(int i=1;i<rows1 * cols1* comp;i=i+3)
                {
                  G[k] =  static_cast<float>(data[i]);
                  k=k+1;
                }
                k=0;
                for(int i=2;i<rows1 * cols1* comp;i=i+3)
                {
                  B[k] =  static_cast<float>(data[i]);
                  k=k+1;
                }


                memcpy(h_A, R, rows1*cols1 * sizeof(float));
                memcpy(h_A +rows1*cols1 , G, rows1*cols1 * sizeof(float));
                memcpy(h_A+rows1*cols1+rows1*cols1, B, rows1*cols1 * sizeof(float));

                memcpy(h_images + img_index * rows1 * cols1 * 3,                     R, rows1 * cols1 * sizeof(float));
                memcpy(h_images + img_index * rows1 * cols1 * 3 + rows1 * cols1,     G, rows1 * cols1 * sizeof(float));
                memcpy(h_images + img_index * rows1 * cols1 * 3 + rows1 * cols1 * 2, B, rows1 * cols1 * sizeof(float));

                img_index++;

             }
             else {
                // Failed to load the image
                std::cerr << "Failed to load image: " << filePath << std::endl;
            }

           }
      }


      float *d_images_temp = (float*)malloc(sizeof(float) * batch_size * rows1 * cols1 * comp);
      for(int i=0; i<batch_size; i++){
        memcpy(d_images_temp + i * rows1 * cols1 *comp , h_images + i * rows1 * cols1 *comp, rows1 * cols1 *comp * sizeof(float));
      }


    float *d_images;
    cudaMalloc((void**)&d_images, batch_size * rows1 * cols1 * comp * sizeof(float));
    cudaMemcpy(d_images, d_images_temp, batch_size * rows1 * cols1 * comp * sizeof(float), cudaMemcpyHostToDevice);

    h_B =(float*)malloc(sizeof(float) * maskdim* maskdim*comp);

    float *d_outs;
    cudaMalloc((void**)&d_outs, batch_size * rows1 * cols1 * sizeof(float));

    float *h_outs = (float*)malloc(sizeof(float) * batch_size * rows1 * cols1);

    float *h_R, *h_G,*h_BB;
    h_R =(float*)malloc(sizeof(float) * maskdim* maskdim);
    h_G =(float*)malloc(sizeof(float) * maskdim* maskdim);
    h_BB =(float*)malloc(sizeof(float) * maskdim* maskdim);
    // Read mask elements
    for (int i = 0; i < maskdim; ++i) {
        for (int j = 0; j < maskdim; ++j) {

            int index =  i * maskdim + j;
            inputFile >>  h_R[index];
            h_R[index]=h_R[index];
            h_G[index]=h_R[index];
            h_BB[index]=h_R[index];

        }
    }

    memcpy(h_B, h_R,  maskdim* maskdim * sizeof(float));
    memcpy(h_B + maskdim* maskdim , h_G, maskdim* maskdim * sizeof(float));
    memcpy(h_B+maskdim* maskdim+maskdim* maskdim,  h_BB, maskdim* maskdim * sizeof(float));


    // Copy data from host to device
    //cudaMemcpyToSymbol(mask, h_B, maskdim * maskdim *comp * sizeof(float), 0, cudaMemcpyHostToDevice);

    float * d_B ;
    cudaMalloc((void**)&d_B, maskdim *  maskdim *comp * sizeof(float));
    cudaMemcpy(d_B, h_B,   maskdim *  maskdim *comp * sizeof(float), cudaMemcpyHostToDevice);

    // Kernel invocation

    dim3 threadsPerBlock(32,32,1);

    int tile_x = threadsPerBlock.x - (maskdim - 1);
    int tile_y = threadsPerBlock.y - (maskdim - 1);
    //int tile_z = threadsPerBlock.z - (batch_size - 1);
    //int tile_z = threadsPerBlock.z;

    int x_blocks = (cols1 + tile_x - 1) / tile_x;
    int y_blocks = (rows1 + tile_y - 1) / tile_y;
    // int z_blocks = (batch_size + tile_z - 1) / tile_z;
    int z_blocks = batch_size;

    int shared_width = tile_x + maskdim - 1;
    int shared_height = tile_y + maskdim - 1;
    //int shared_depth = tile_z + batch_size - 1;
    int shared_depth = batch_size;

    dim3 numBlocks(x_blocks, y_blocks, z_blocks);

    Imgcov3d<<<numBlocks, threadsPerBlock,(sizeof(float)*shared_width*shared_height*3*shared_depth)>>>(batch_size,d_images,d_outs,cols1,rows1,d_B, maskdim,shared_width,shared_height,shared_depth);

    cudaMemcpy(h_outs, d_outs, batch_size * rows1 * cols1 * sizeof(float), cudaMemcpyDeviceToHost);

    unsigned char* h_char = (unsigned char*)malloc(sizeof(unsigned char) * rows1 * cols1);

    for(int k=0;k<batch_size;k++){

      for(int i=0;i<rows1*cols1;i++){
         h_char[i]=static_cast<unsigned char>(h_outs[k*rows1*cols1+i]);
      }
      string filename = "./"+out_path+"/"+"image" + std::to_string(k) + ".jpg";
      stbi_write_jpg(filename.c_str(), cols1, rows1, 1, h_char, 100);
    }

    // Free device memory
    cudaFree(d_images);
    cudaFree(d_B);
    cudaFree(d_outs);

    // Free host memory
    free(h_A);
    free(h_B);
    free(R);
    free(G);
    free(B);
    free(h_images);
    free(d_images_temp);
    free(h_outs);
    free(h_BB);
    free(h_G);
    free(h_R);
    free(h_char);

  return 0;
}

Overwriting K2_2_31_2_33.cu


In [89]:
!chmod +x K2_2_31_2_33.cu
!nvcc  K2_2_31_2_33.cu -o K2_2_31_2_33

In [90]:
!nvprof ./K2_2_31_2_33 "in_255" "out2" 4 "mask_3.txt"

maskdim 3
cols 255
rows 255
image in_255/you.jpg
cols 255
rows 255
image in_255/warda.jpg
cols 255
rows 255
image in_255/cat.jpg
cols 255
rows 255
image in_255/camel.jpg
==18493== NVPROF is profiling process 18493, command: ./K2_2_31_2_33 in_255 out2 4 mask_3.txt
==18493== Profiling application: ./K2_2_31_2_33 in_255 out2 4 mask_3.txt
==18493== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   71.94%  514.29us         2  257.15us     992ns  513.30us  [CUDA memcpy HtoD]
                   16.37%  117.02us         1  117.02us  117.02us  117.02us  [CUDA memcpy DtoH]
                   11.69%  83.581us         1  83.581us  83.581us  83.581us  Imgcov3d(int, float*, float*, unsigned int, unsigned int, float const *, int, int, int, int)
      API calls:   97.67%  101.00ms         3  33.666ms  23.948us  100.86ms  cudaMalloc
                    1.63%  1.6903ms         3  563.42us  12.298us  976.95us  cudaMemcpy
               

In [91]:
!nvprof ./K2_2_31_2_33 "in_255" "out2" 4 "mask_5.txt"

maskdim 5
cols 255
rows 255
image in_255/you.jpg
cols 255
rows 255
image in_255/warda.jpg
cols 255
rows 255
image in_255/cat.jpg
cols 255
rows 255
image in_255/camel.jpg
==18616== NVPROF is profiling process 18616, command: ./K2_2_31_2_33 in_255 out2 4 mask_5.txt
==18616== Profiling application: ./K2_2_31_2_33 in_255 out2 4 mask_5.txt
==18616== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   63.10%  520.37us         2  260.19us     992ns  519.38us  [CUDA memcpy HtoD]
                   23.46%  193.50us         1  193.50us  193.50us  193.50us  Imgcov3d(int, float*, float*, unsigned int, unsigned int, float const *, int, int, int, int)
                   13.44%  110.81us         1  110.81us  110.81us  110.81us  [CUDA memcpy DtoH]
      API calls:   96.60%  97.806ms         3  32.602ms  4.9560us  97.673ms  cudaMalloc
                    1.76%  1.7789ms         3  592.97us  12.601us  1.0951ms  cudaMemcpy
               

In [92]:
!nvprof ./K2_2_31_2_33 "in_255" "out2" 4 "mask_7.txt"

maskdim 7
cols 255
rows 255
image in_255/you.jpg
cols 255
rows 255
image in_255/warda.jpg
cols 255
rows 255
image in_255/cat.jpg
cols 255
rows 255
image in_255/camel.jpg
==18747== NVPROF is profiling process 18747, command: ./K2_2_31_2_33 in_255 out2 4 mask_7.txt
==18747== Profiling application: ./K2_2_31_2_33 in_255 out2 4 mask_7.txt
==18747== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   54.06%  519.25us         2  259.63us  1.0240us  518.23us  [CUDA memcpy HtoD]
                   33.55%  322.23us         1  322.23us  322.23us  322.23us  Imgcov3d(int, float*, float*, unsigned int, unsigned int, float const *, int, int, int, int)
                   12.38%  118.94us         1  118.94us  118.94us  118.94us  [CUDA memcpy DtoH]
      API calls:   97.16%  98.055ms         3  32.685ms  7.5900us  97.921ms  cudaMalloc
                    2.03%  2.0442ms         3  681.41us  13.179us  1.2962ms  cudaMemcpy
               

In [93]:
!nvprof ./K2_2_31_2_33 "in_255" "out2" 4 "mask_9.txt"

maskdim 9
cols 255
rows 255
image in_255/you.jpg
cols 255
rows 255
image in_255/warda.jpg
cols 255
rows 255
image in_255/cat.jpg
cols 255
rows 255
image in_255/camel.jpg
==18830== NVPROF is profiling process 18830, command: ./K2_2_31_2_33 in_255 out2 4 mask_9.txt
==18830== Profiling application: ./K2_2_31_2_33 in_255 out2 4 mask_9.txt
==18830== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   49.52%  606.16us         1  606.16us  606.16us  606.16us  Imgcov3d(int, float*, float*, unsigned int, unsigned int, float const *, int, int, int, int)
                   43.77%  535.73us         2  267.86us  1.0230us  534.71us  [CUDA memcpy HtoD]
                    6.71%  82.078us         1  82.078us  82.078us  82.078us  [CUDA memcpy DtoH]
      API calls:   97.14%  97.382ms         3  32.461ms  5.6180us  97.266ms  cudaMalloc
                    2.13%  2.1368ms         3  712.28us  12.573us  1.4229ms  cudaMemcpy
               

In [94]:
!nvprof ./K2_2_31_2_33 "in_255" "out2" 4 "mask_11.txt"

maskdim 11
cols 255
rows 255
image in_255/you.jpg
cols 255
rows 255
image in_255/warda.jpg
cols 255
rows 255
image in_255/cat.jpg
cols 255
rows 255
image in_255/camel.jpg
==18905== NVPROF is profiling process 18905, command: ./K2_2_31_2_33 in_255 out2 4 mask_11.txt
==18905== Profiling application: ./K2_2_31_2_33 in_255 out2 4 mask_11.txt
==18905== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   59.56%  942.69us         1  942.69us  942.69us  942.69us  Imgcov3d(int, float*, float*, unsigned int, unsigned int, float const *, int, int, int, int)
                   35.28%  558.35us         2  279.18us  1.0240us  557.33us  [CUDA memcpy HtoD]
                    5.16%  81.662us         1  81.662us  81.662us  81.662us  [CUDA memcpy DtoH]
      API calls:   96.46%  88.550ms         3  29.517ms  5.0580us  88.418ms  cudaMalloc
                    2.71%  2.4834ms         3  827.81us  12.458us  1.7363ms  cudaMemcpy
            

In [95]:
!nvprof ./K2_2_31_2_33 "in_255" "out2" 4 "mask_13.txt"

maskdim 13
cols 255
rows 255
image in_255/you.jpg
cols 255
rows 255
image in_255/warda.jpg
cols 255
rows 255
image in_255/cat.jpg
cols 255
rows 255
image in_255/camel.jpg
==18978== NVPROF is profiling process 18978, command: ./K2_2_31_2_33 in_255 out2 4 mask_13.txt
==18978== Profiling application: ./K2_2_31_2_33 in_255 out2 4 mask_13.txt
==18978== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   67.96%  1.3419ms         1  1.3419ms  1.3419ms  1.3419ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, float const *, int, int, int, int)
                   26.96%  532.27us         2  266.14us  1.0880us  531.19us  [CUDA memcpy HtoD]
                    5.09%  100.48us         1  100.48us  100.48us  100.48us  [CUDA memcpy DtoH]
      API calls:   95.95%  87.991ms         3  29.330ms  7.3590us  87.856ms  cudaMalloc
                    3.25%  2.9773ms         3  992.43us  12.713us  2.2717ms  cudaMemcpy
            

In [96]:
!nvprof ./K2_2_31_2_33 "in_255" "out2" 4 "mask_15.txt"

maskdim 15
cols 255
rows 255
image in_255/you.jpg
cols 255
rows 255
image in_255/warda.jpg
cols 255
rows 255
image in_255/cat.jpg
cols 255
rows 255
image in_255/camel.jpg
==19061== NVPROF is profiling process 19061, command: ./K2_2_31_2_33 in_255 out2 4 mask_15.txt
==19061== Profiling application: ./K2_2_31_2_33 in_255 out2 4 mask_15.txt
==19061== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   77.74%  2.1607ms         1  2.1607ms  2.1607ms  2.1607ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, float const *, int, int, int, int)
                   18.71%  519.95us         2  259.98us  1.1520us  518.80us  [CUDA memcpy HtoD]
                    3.55%  98.589us         1  98.589us  98.589us  98.589us  [CUDA memcpy DtoH]
      API calls:   95.71%  101.02ms         3  33.674ms  5.0740us  100.90ms  cudaMalloc
                    3.59%  3.7912ms         3  1.2637ms  13.240us  3.0695ms  cudaMemcpy
            

In [99]:
!nvprof ./K2_2_31_2_33 "in_255" "out2" 4 "mask_edge_5.txt"

maskdim 5
cols 255
rows 255
image in_255/you.jpg
cols 255
rows 255
image in_255/warda.jpg
cols 255
rows 255
image in_255/cat.jpg
cols 255
rows 255
image in_255/camel.jpg
==19494== NVPROF is profiling process 19494, command: ./K2_2_31_2_33 in_255 out2 4 mask_edge_5.txt
==19494== Profiling application: ./K2_2_31_2_33 in_255 out2 4 mask_edge_5.txt
==19494== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   68.70%  656.88us         2  328.44us  1.0240us  655.85us  [CUDA memcpy HtoD]
                   20.23%  193.44us         1  193.44us  193.44us  193.44us  Imgcov3d(int, float*, float*, unsigned int, unsigned int, float const *, int, int, int, int)
                   11.07%  105.82us         1  105.82us  105.82us  105.82us  [CUDA memcpy DtoH]
      API calls:   97.71%  138.14ms         3  46.045ms  8.2520us  137.97ms  cudaMalloc
                    1.59%  2.2548ms         3  751.59us  17.010us  1.3649ms  cudaMemcpy
     

In [103]:
%%writefile K3_2_31_2_33.cu
//Norhan_Reda_Abdelwahed_2_31
//Hoda_Gamal_Hamouda_2_33

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <iostream>
#include <iomanip>
#include <fstream>
#include <vector>
#define STB_IMAGE_IMPLEMENTATION
#include <stdio.h>
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"
#include <string>
#include <filesystem>
#include <cuda_runtime.h>

using namespace std;


__global__ void Imgcov3d(int batchSize, float *input, float* output, unsigned int width,unsigned int height, const float* __restrict__ mask,int maskdim, int shared_width,int shared_height, int shared_depth){

  extern __shared__ float sharedTile [];

    int tx = threadIdx.x;
    int ty = threadIdx.y;

    int bx = blockIdx.x;
    int by = blockIdx.y;

    int tile_x = shared_width - (maskdim - 1);
    int tile_y = shared_height - (maskdim - 1);

    int x_o = blockIdx.x * tile_x + threadIdx.x;
    int y_o = blockIdx.y * tile_y + threadIdx.y;

    int idx = blockIdx.z * blockDim.z + threadIdx.z;

    if (idx >= batchSize){
      return;
    }

    int num = ((width*height) - 1) /(shared_width*shared_height) + 1;

    for (int d=0; d< 3; d++){
      for(int f=0; f < num ; f++){

          int destTmp = tile_x * tile_y * f + ty * tile_x + tx;

          int dX = destTmp % shared_width;
          destTmp = destTmp / shared_width;
          int dY = destTmp % shared_height;

          int inputX = dX + (bx * tile_x) - (maskdim/2);
          int inputY = dY + (by * tile_y) - (maskdim/2);

          if(inputY >= 0 && inputY < height && inputX >= 0 && inputX < width){
                sharedTile[idx * shared_width * shared_height * 3 + d * shared_height * shared_width + dY * shared_width + dX]
                = input[idx * width * height * 3 + d * height * width + inputY * width + inputX];
          }
          else{
              sharedTile[idx * shared_width * shared_height * 3 + d * shared_height * shared_width + dY * shared_width + dX] = 0.0;
          }
      }
    }

    __syncthreads();

  float sum = 0;

    for(int y_mask = 0; y_mask < maskdim; y_mask++) {
            for(int x_mask = 0; x_mask < maskdim; x_mask++) {
                  for(int z_mask = 0; z_mask < 3; z_mask++) {
                    sum += mask[z_mask * maskdim * maskdim + y_mask * maskdim + x_mask] *
                    sharedTile[idx * shared_height * shared_width * 3 + z_mask * shared_height * shared_width + (ty + y_mask) * shared_width +(tx + x_mask)];

                }
             }
         }

    if(sum < 0){
      sum = 0;
    } else if (sum > 255){
      sum = 255;
    }

    if(y_o < height && x_o < width){
        output[idx * width * height + y_o * width + x_o] = sum;
    }
}

int main(int argc, char *argv[]) {

    string folderPath = argv[1];
    string out_path = argv[2];
    int batch_size = stoi(argv[3]);
    string mask_path = argv[4];

    int maskdim;

    // Open the file for reading
    std::ifstream inputFile(mask_path);

    // Check if the file is opened successfully
    if (!inputFile.is_open()) {
        std::cerr << "Error opening file: " <<mask_path  << std::endl;
        return 1; // Exit with an error code
    }

     inputFile >> maskdim ;
     cout<<"maskdim "<<maskdim<<endl;

    float *h_A, *h_B,*R,*G,*B;
    int rows1,cols1, comp;

    float *h_images;
    int img_index = 0;

    for (const auto& entry : std::filesystem::directory_iterator(folderPath)) {

        if (entry.is_regular_file()) {
            std::string filePath = entry.path().string();

            unsigned char *data = stbi_load(filePath.c_str(), &cols1, &rows1, &comp, 0);

            if(img_index == 0){
              h_images = (float*)malloc(sizeof(float) * batch_size * cols1 * rows1 * comp);
            }

            if (data) {

                printf("cols %d\n", cols1);
                printf("rows %d\n", rows1);
                printf("image %s\n", filePath.c_str());

                h_A = (float*)malloc(sizeof(float) *  rows1 * cols1* comp);
                R = (float*)malloc(sizeof(float) *  rows1 * cols1);
                G = (float*)malloc(sizeof(float) *  rows1 * cols1);
                B = (float*)malloc(sizeof(float) *  rows1 * cols1);

                int k = 0;
                for(int i=0;i<rows1 * cols1* comp;i=i+3)
                {
                  R[k] =  static_cast<float>(data[i]);
                  k=k+1;
                }
                k=0;
                for(int i=1;i<rows1 * cols1* comp;i=i+3)
                {
                  G[k] =  static_cast<float>(data[i]);
                  k=k+1;
                }
                k=0;
                for(int i=2;i<rows1 * cols1* comp;i=i+3)
                {
                  B[k] =  static_cast<float>(data[i]);
                  k=k+1;
                }


                memcpy(h_A, R, rows1*cols1 * sizeof(float));
                memcpy(h_A +rows1*cols1 , G, rows1*cols1 * sizeof(float));
                memcpy(h_A+rows1*cols1+rows1*cols1, B, rows1*cols1 * sizeof(float));

                memcpy(h_images + img_index * rows1 * cols1 * 3,                     R, rows1 * cols1 * sizeof(float));
                memcpy(h_images + img_index * rows1 * cols1 * 3 + rows1 * cols1,     G, rows1 * cols1 * sizeof(float));
                memcpy(h_images + img_index * rows1 * cols1 * 3 + rows1 * cols1 * 2, B, rows1 * cols1 * sizeof(float));

                img_index++;

             }
             else {
                // Failed to load the image
                std::cerr << "Failed to load image: " << filePath << std::endl;
            }

           }
      }


      float *d_images_temp = (float*)malloc(sizeof(float) * batch_size * rows1 * cols1 * comp);
      for(int i=0; i<batch_size; i++){
        memcpy(d_images_temp + i * rows1 * cols1 *comp , h_images + i * rows1 * cols1 *comp, rows1 * cols1 *comp * sizeof(float));
      }


    float *d_images;
    cudaMalloc((void**)&d_images, batch_size * rows1 * cols1 * comp * sizeof(float));
    cudaMemcpy(d_images, d_images_temp, batch_size * rows1 * cols1 * comp * sizeof(float), cudaMemcpyHostToDevice);

    h_B =(float*)malloc(sizeof(float) * maskdim* maskdim*comp);

    float *d_outs;
    cudaMalloc((void**)&d_outs, batch_size * rows1 * cols1 * sizeof(float));

    float *h_outs = (float*)malloc(sizeof(float) * batch_size * rows1 * cols1);

    float *h_R, *h_G,*h_BB;
    h_R =(float*)malloc(sizeof(float) * maskdim* maskdim);
    h_G =(float*)malloc(sizeof(float) * maskdim* maskdim);
    h_BB =(float*)malloc(sizeof(float) * maskdim* maskdim);
    // Read mask elements
    for (int i = 0; i < maskdim; ++i) {
        for (int j = 0; j < maskdim; ++j) {

            int index =  i * maskdim + j;
            inputFile >>  h_R[index];
            h_R[index]=h_R[index];
            h_G[index]=h_R[index];
            h_BB[index]=h_R[index];

        }
    }

    memcpy(h_B, h_R,  maskdim* maskdim * sizeof(float));
    memcpy(h_B + maskdim* maskdim , h_G, maskdim* maskdim * sizeof(float));
    memcpy(h_B+maskdim* maskdim+maskdim* maskdim,  h_BB, maskdim* maskdim * sizeof(float));


    // Copy data from host to device
    //cudaMemcpyToSymbol(mask, h_B, maskdim * maskdim *comp * sizeof(float), 0, cudaMemcpyHostToDevice);

    float * d_B ;
    cudaMalloc((void**)&d_B, maskdim *  maskdim *comp * sizeof(float));
    cudaMemcpy(d_B, h_B,   maskdim *  maskdim *comp * sizeof(float), cudaMemcpyHostToDevice);

    // Kernel invocation

    dim3 threadsPerBlock(16,16,3);

    int tile_x = threadsPerBlock.x;
    int tile_y = threadsPerBlock.y;
    int tile_z = threadsPerBlock.z;

    int x_blocks = (cols1 + tile_x - 1) / tile_x;
    int y_blocks = (rows1 + tile_y - 1) / tile_y;
   int z_blocks = (batch_size + tile_z - 1) / tile_z;
  // int z_blocks = batch_size;

    int shared_width = tile_x + maskdim - 1;
    int shared_height = tile_y + maskdim - 1;
    //int shared_depth = tile_z + batch_size - 1;
    int shared_depth = batch_size;

    dim3 numBlocks(x_blocks, y_blocks, z_blocks);

    Imgcov3d<<<numBlocks, threadsPerBlock,(sizeof(float)*shared_width*shared_height*3*shared_depth)>>>(batch_size,d_images,d_outs,cols1,rows1,d_B, maskdim,shared_width,shared_height,shared_depth);

    cudaMemcpy(h_outs, d_outs, batch_size * rows1 * cols1 * sizeof(float), cudaMemcpyDeviceToHost);

    unsigned char* h_char = (unsigned char*)malloc(sizeof(unsigned char) * rows1 * cols1);

    for(int k=0;k<batch_size;k++){

      for(int i=0;i<rows1*cols1;i++){
         h_char[i]=static_cast<unsigned char>(h_outs[k*rows1*cols1+i]);
      }
      string filename = "./"+out_path+"/"+"image" + std::to_string(k) + ".jpg";
      stbi_write_jpg(filename.c_str(), cols1, rows1, 1, h_char, 100);
    }

    // Free device memory
    cudaFree(d_images);
    cudaFree(d_B);
    cudaFree(d_outs);

    // Free host memory
    free(h_A);
    free(h_B);
    free(R);
    free(G);
    free(B);
    free(h_images);
    free(d_images_temp);
    free(h_outs);
    free(h_BB);
    free(h_G);
    free(h_R);
    free(h_char);

  return 0;
}

Overwriting K3_2_31_2_33.cu


In [104]:
!chmod +x K3_2_31_2_33.cu
!nvcc  K3_2_31_2_33.cu -o K3_2_31_2_33

In [105]:
!nvprof ./K3_2_31_2_33 "in_255" "out3" 4 "mask_3.txt"

maskdim 3
cols 255
rows 255
image in_255/you.jpg
cols 255
rows 255
image in_255/warda.jpg
cols 255
rows 255
image in_255/cat.jpg
cols 255
rows 255
image in_255/camel.jpg
==20253== NVPROF is profiling process 20253, command: ./K3_2_31_2_33 in_255 out3 4 mask_3.txt
==20253== Profiling application: ./K3_2_31_2_33 in_255 out3 4 mask_3.txt
==20253== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   86.39%  4.0209ms         1  4.0209ms  4.0209ms  4.0209ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, float const *, int, int, int, int)
                   11.83%  550.48us         2  275.24us     992ns  549.49us  [CUDA memcpy HtoD]
                    1.78%  82.750us         1  82.750us  82.750us  82.750us  [CUDA memcpy DtoH]
      API calls:   93.66%  94.295ms         3  31.432ms  5.3880us  94.168ms  cudaMalloc
                    5.56%  5.5933ms         3  1.8644ms  12.643us  4.8355ms  cudaMemcpy
               

In [106]:
!nvprof ./K3_2_31_2_33 "in_255" "out3" 4 "mask_5.txt"

maskdim 5
cols 255
rows 255
image in_255/you.jpg
cols 255
rows 255
image in_255/warda.jpg
cols 255
rows 255
image in_255/cat.jpg
cols 255
rows 255
image in_255/camel.jpg
==20360== NVPROF is profiling process 20360, command: ./K3_2_31_2_33 in_255 out3 4 mask_5.txt
==20360== Profiling application: ./K3_2_31_2_33 in_255 out3 4 mask_5.txt
==20360== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   84.63%  3.4627ms         1  3.4627ms  3.4627ms  3.4627ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, float const *, int, int, int, int)
                   13.04%  533.55us         2  266.78us     992ns  532.56us  [CUDA memcpy HtoD]
                    2.33%  95.165us         1  95.165us  95.165us  95.165us  [CUDA memcpy DtoH]
      API calls:   94.48%  102.37ms         3  34.122ms  6.4880us  102.23ms  cudaMalloc
                    4.75%  5.1420ms         3  1.7140ms  14.198us  4.4102ms  cudaMemcpy
               

In [107]:
!nvprof ./K3_2_31_2_33 "in_255" "out3" 4 "mask_7.txt"

maskdim 7
cols 255
rows 255
image in_255/you.jpg
cols 255
rows 255
image in_255/warda.jpg
cols 255
rows 255
image in_255/cat.jpg
cols 255
rows 255
image in_255/camel.jpg
==20429== NVPROF is profiling process 20429, command: ./K3_2_31_2_33 in_255 out3 4 mask_7.txt
==20429== Profiling application: ./K3_2_31_2_33 in_255 out3 4 mask_7.txt
==20429== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   82.91%  3.0417ms         1  3.0417ms  3.0417ms  3.0417ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, float const *, int, int, int, int)
                   14.22%  521.78us         2  260.89us  1.0240us  520.75us  [CUDA memcpy HtoD]
                    2.87%  105.34us         1  105.34us  105.34us  105.34us  [CUDA memcpy DtoH]
      API calls:   94.73%  97.451ms         3  32.484ms  11.558us  97.318ms  cudaMalloc
                    4.53%  4.6569ms         3  1.5523ms  12.859us  3.9380ms  cudaMemcpy
               

In [108]:
!nvprof ./K3_2_31_2_33 "in_255" "out3" 4 "mask_9.txt"

maskdim 9
cols 255
rows 255
image in_255/you.jpg
cols 255
rows 255
image in_255/warda.jpg
cols 255
rows 255
image in_255/cat.jpg
cols 255
rows 255
image in_255/camel.jpg
==20504== NVPROF is profiling process 20504, command: ./K3_2_31_2_33 in_255 out3 4 mask_9.txt
==20504== Profiling application: ./K3_2_31_2_33 in_255 out3 4 mask_9.txt
==20504== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   80.76%  2.7943ms         1  2.7943ms  2.7943ms  2.7943ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, float const *, int, int, int, int)
                   15.83%  547.73us         2  273.86us  1.0560us  546.67us  [CUDA memcpy HtoD]
                    3.41%  118.05us         1  118.05us  118.05us  118.05us  [CUDA memcpy DtoH]
      API calls:   94.56%  91.980ms         3  30.660ms  5.5810us  91.855ms  cudaMalloc
                    4.57%  4.4496ms         3  1.4832ms  12.658us  3.6932ms  cudaMemcpy
               

In [109]:
!nvprof ./K3_2_31_2_33 "in_255" "out3" 4 "mask_11.txt"

maskdim 11
cols 255
rows 255
image in_255/you.jpg
cols 255
rows 255
image in_255/warda.jpg
cols 255
rows 255
image in_255/cat.jpg
cols 255
rows 255
image in_255/camel.jpg
==20595== NVPROF is profiling process 20595, command: ./K3_2_31_2_33 in_255 out3 4 mask_11.txt
==20595== Profiling application: ./K3_2_31_2_33 in_255 out3 4 mask_11.txt
==20595== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   79.98%  2.7074ms         1  2.7074ms  2.7074ms  2.7074ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, float const *, int, int, int, int)
                   16.49%  558.23us         2  279.11us  1.0560us  557.17us  [CUDA memcpy HtoD]
                    3.53%  119.61us         1  119.61us  119.61us  119.61us  [CUDA memcpy DtoH]
      API calls:   94.69%  90.348ms         3  30.116ms  5.2850us  90.227ms  cudaMalloc
                    4.50%  4.2970ms         3  1.4323ms  12.161us  3.5695ms  cudaMemcpy
            

In [110]:
!nvprof ./K3_2_31_2_33 "in_255" "out3" 4 "mask_13.txt"

maskdim 13
cols 255
rows 255
image in_255/you.jpg
cols 255
rows 255
image in_255/warda.jpg
cols 255
rows 255
image in_255/cat.jpg
cols 255
rows 255
image in_255/camel.jpg
==20660== NVPROF is profiling process 20660, command: ./K3_2_31_2_33 in_255 out3 4 mask_13.txt
==20660== Profiling application: ./K3_2_31_2_33 in_255 out3 4 mask_13.txt
==20660== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   81.57%  2.7328ms         1  2.7328ms  2.7328ms  2.7328ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, float const *, int, int, int, int)
                   15.66%  524.69us         2  262.35us  1.0880us  523.60us  [CUDA memcpy HtoD]
                    2.77%  92.861us         1  92.861us  92.861us  92.861us  [CUDA memcpy DtoH]
      API calls:   94.59%  90.134ms         3  30.045ms  8.2030us  89.996ms  cudaMalloc
                    4.58%  4.3676ms         3  1.4559ms  14.528us  3.5955ms  cudaMemcpy
            

In [111]:
!nvprof ./K3_2_31_2_33 "in_255" "out3" 4 "mask_15.txt"

maskdim 15
cols 255
rows 255
image in_255/you.jpg
cols 255
rows 255
image in_255/warda.jpg
cols 255
rows 255
image in_255/cat.jpg
cols 255
rows 255
image in_255/camel.jpg
==20741== NVPROF is profiling process 20741, command: ./K3_2_31_2_33 in_255 out3 4 mask_15.txt
==20741== Profiling application: ./K3_2_31_2_33 in_255 out3 4 mask_15.txt
==20741== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   82.64%  2.9154ms         1  2.9154ms  2.9154ms  2.9154ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, float const *, int, int, int, int)
                   14.81%  522.61us         2  261.30us  1.1520us  521.46us  [CUDA memcpy HtoD]
                    2.55%  89.789us         1  89.789us  89.789us  89.789us  [CUDA memcpy DtoH]
      API calls:   94.38%  87.849ms         3  29.283ms  4.8390us  87.723ms  cudaMalloc
                    4.77%  4.4428ms         3  1.4809ms  12.903us  3.7314ms  cudaMemcpy
            

In [112]:
!nvprof ./K3_2_31_2_33 "in_255" "out3" 4 "mask_edge_5.txt"

maskdim 5
cols 255
rows 255
image in_255/you.jpg
cols 255
rows 255
image in_255/warda.jpg
cols 255
rows 255
image in_255/cat.jpg
cols 255
rows 255
image in_255/camel.jpg
==20818== NVPROF is profiling process 20818, command: ./K3_2_31_2_33 in_255 out3 4 mask_edge_5.txt
==20818== Profiling application: ./K3_2_31_2_33 in_255 out3 4 mask_edge_5.txt
==20818== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   84.07%  3.4645ms         1  3.4645ms  3.4645ms  3.4645ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, float const *, int, int, int, int)
                   13.05%  537.68us         2  268.84us  1.0240us  536.66us  [CUDA memcpy HtoD]
                    2.88%  118.72us         1  118.72us  118.72us  118.72us  [CUDA memcpy DtoH]
      API calls:   94.02%  91.219ms         3  30.406ms  7.0110us  91.093ms  cudaMalloc
                    5.21%  5.0585ms         3  1.6862ms  12.815us  4.3432ms  cudaMemcpy
     

# batches

In [45]:
!pip install torch torchvision

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [46]:
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
import cv2
import time
import numpy as np
import os
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

# Define the mask tensor
file_path = "mask.txt"

# Open the file in read mode
with open(file_path, "r") as file:
    # Read the lines of the file
    lines = file.readlines()

# Remove any leading or trailing whitespace characters from each line
lines = [line.strip() for line in lines]

# Extract the first line and assign it to a variable
MASKDIM = int(lines[0])
print("maskdim", MASKDIM)
# Extract the remaining lines and create a matrix
mask = [(line.split()) for line in lines[1:]]
# Convert the list to a numpy array
array_2d = np.array(mask, dtype=float)

mask = np.repeat(array_2d[np.newaxis, ...], 3, axis=0)

print(mask)

# Process the variables
print("MASKDIM:", MASKDIM)
print("Matrix:")
# for row in mask:
#     print(row)
print("mask shape",mask.shape)

# Convert the mask to a PyTorch tensor
mask = torch.from_numpy(mask)

print(mask)

mask = mask.unsqueeze(0).unsqueeze(0)[:,:3,:,:,:]
mask = mask.float()

# Folder path containing the images
input_path = "in"
output_path = "output"
# List all image file names in the folder
image_files = os.listdir(input_path)

imageShape0 = None
imageShape1 = None
# Define the allowed image formats
allowed_formats = (".jpg", ".jpeg", ".png")
# Create an empty list to store the preprocessed images
image_list = []
# Iterate over the image files
for image_file in image_files:
   # Check if the file has one of the allowed formats
  if image_file.lower().endswith(allowed_formats):
    print(image_file)
    # Read the RGB image using OpenCV
    image = cv2.imread(os.path.join(input_path, image_file))
    imageShape0 =  image.shape[0]
    imageShape1 =  image.shape[1]
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    print("image file",image_file)

    # print(image[0])
    image_array = np.array(image)
    max_value = np.max(image_array)
    max_value_arg = np.argmax(image_array)
    # print('===>', max_value, max_value_arg)

    # Define the padding size
    padding = (MASKDIM - 1) // 2
    print(padding)

    pad_height = (padding, padding)  # Pad 2 pixels on each side along height
    pad_width = (padding, padding)  # Pad 2 pixels on each side along width
    pad_channels = (0, 0)  # No padding along channels (RGB)

    # Pad the image
    padded_image = np.pad(image, (pad_height, pad_width, pad_channels), mode='constant', constant_values=0)  # Pad with zeros
    print(padded_image.shape)
    print("Data type of input_image:", padded_image.dtype)
    # Convert the image to a PyTorch tensor
    transform = transforms.ToTensor()
    input_image = transform(padded_image)
    print(input_image.shape)
    print("Data type of input_image:", input_image.dtype)

    input_image = input_image * 255

    # Append the tensor to the image list
    image_list.append(input_image)

# Stack the image tensors into a single tensor along the batch dimension
batch_images = torch.stack(image_list, dim=0)

# # Reshape the tensor to match the expected shape (batch_size, channels, depth, height, width)
# batch_images = batch_images.unsqueeze(2).expand(-1, -1, desired_dims[0], -1, -1)
batch_images = batch_images.unsqueeze(1)
print("batch_images shape",batch_images.shape)
print("mask size",mask.shape)
# Perform the 3D convolution using PyTorch
# Start the timer
start_time = time.time()
output_tensor = F.conv3d(batch_images, mask)
print("output tensor",output_tensor.shape)
# Stop the timer
end_time = time.time()
output_tensor = output_tensor.to(torch.int)
output_tensor = output_tensor.squeeze(2).numpy()
print("output tensor",output_tensor.shape)
print("shape",output_tensor.shape[0])
if not os.path.exists(output_path):
    # Create the folder using os.mkdir()
    os.mkdir(output_path)
    print(f"Folder created at: {output_path}")
else:
    print(f"Folder already exists at: {output_path}")
for i in range(output_tensor.shape[0]):
  output_array = output_tensor[i]
  print(output_array.shape)
  output_array = np.clip(output_array, 0, 255)  # Clip values to the range [0, 255]
  print(output_array)
  cv2.imwrite(output_path+'/'+'image'+str(i)+'.jpg', output_array.reshape(imageShape0 , imageShape1, 1))

True


FileNotFoundError: [Errno 2] No such file or directory: 'mask.txt'

In [78]:
!python pytorch.py "in_255" "out_py" "4" "mask_3.txt"

Folder already exists at: out_py
execution time 0.01511836051940918


In [79]:
!python pytorch.py "in_255" "out_py" "4" "mask_5.txt"

Folder already exists at: out_py
execution time 0.019618988037109375


In [80]:
!python pytorch.py "in_255" "out_py" "4" "mask_7.txt"

Folder already exists at: out_py
execution time 0.024872303009033203


In [81]:
!python pytorch.py "in_255" "out_py" "4" "mask_9.txt"

Folder already exists at: out_py
execution time 0.042836666107177734


In [82]:
!python pytorch.py "in_255" "out_py" "4" "mask_11.txt"

Folder already exists at: out_py
execution time 0.03837013244628906


In [83]:
!python pytorch.py "in_255" "out_py" "4" "mask_13.txt"

Folder already exists at: out_py
execution time 0.04645037651062012


In [84]:
!python pytorch.py "in_255" "out_py" "4" "mask_15.txt"

Folder already exists at: out_py
execution time 0.062044382095336914


In [85]:
!python pytorch.py "in_255" "out_py" "4" "mask_edge_5.txt"

Folder already exists at: out_py
execution time 0.019329547882080078
